In [ ]:
from google import colab
!pip install catboost
!pip install optuna
from sklearn.model_selection import StratifiedKFold,train_test_split
import optuna
from optuna.samplers import TPESampler
import tensorflow as tf
import random
import os
import sys
import platform
import random
import math
from typing import List ,Dict, Tuple

import pandas as pd
import numpy as np
 
import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

from catboost import Pool,CatBoostClassifier
colab.drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = "/content/drive/MyDrive/Jobcare_data/"
SEED = 42

# 매번 모델링을 할 때마다 동일한 결과를 얻기 위해 랜덤 시드 설정 동일하게 유지
# 럭키 시드 찾는건 시간상 어려움

def reset_seeds(reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(1)
    random.seed(2)
    tf.compat.v1.set_random_seed(3)
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # for GPU
    print("RANDOM SEEDS RESET")  # optional
   
reset_seeds()

RANDOM SEEDS RESET


In [ ]:
print(f"- os: {platform.platform()}")
print(f"- python: {sys.version}")
print(f"- pandas: {pd.__version__}")
print(f"- numpy: {np.__version__}")
print(f"- sklearn: {sklearn.__version__}")

- os: Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
- python: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
- pandas: 1.1.5
- numpy: 1.19.5
- sklearn: 1.0.2


In [ ]:
train_data = pd.read_csv(f'{DATA_PATH}train.csv').sort_values(by="contents_open_dt")
test_data = pd.read_csv(f'{DATA_PATH}test.csv')

code_d = pd.read_csv(f'{DATA_PATH}속성_D_코드.csv')
code_h = pd.read_csv(f'{DATA_PATH}속성_H_코드.csv')
code_l = pd.read_csv(f'{DATA_PATH}속성_L_코드.csv')

train_data.shape , test_data.shape

((501951, 35), (46404, 34))

In [ ]:
code_d.columns= ["attribute_d","attribute_d_d","attribute_d_s","attribute_d_m","attribute_d_l"]
code_h.columns= ["attribute_h","attribute_h_m","attribute_h_l"]
code_l.columns= ["attribute_l","attribute_l_d","attribute_l_s","attribute_l_m","attribute_l_l"]

In [ ]:
def merge_codes(df:pd.DataFrame,df_code:pd.DataFrame,col:str)->pd.DataFrame:
    df = df.copy()
    df_code = df_code.copy()
    df_code = df_code.add_prefix(f"{col}_")
    df_code.columns.values[0] = col
    return pd.merge(df,df_code,how="left",on=col)

In [ ]:
def preprocess_data(
                    df:pd.DataFrame,is_train:bool = True, cols_merge:List[Tuple[str,pd.DataFrame]] = []  , cols_equi:List[Tuple[str,str]]= [] ,
                    cols_drop:List[str] = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt"]
                    )->Tuple[pd.DataFrame,np.ndarray]:
    df = df.copy()

    y_data = None
    if is_train:
        y_data = df["target"].to_numpy()
        df = df.drop(columns="target")

    for col, df_code in cols_merge:
        df = merge_codes(df,df_code,col)

    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] = df[cols].astype(int)

    for col1, col2 in cols_equi:
        df[f"{col1}_match_{col2}"] = (df[col1] == df[col2] ).astype(int)

    df = df.drop(columns=cols_drop)
    return (df , y_data)

In [ ]:
# 소분류 중분류 대분류 속성코드 merge 컬럼명 및 데이터 프레임 리스트
cols_merge = [
              ("person_prefer_d_1" , code_d),
              ("person_prefer_d_2" , code_d),
              ("person_prefer_d_3" , code_d),
              ("contents_attribute_d" , code_d),
              ("person_prefer_h_1" , code_h),
              ("person_prefer_h_2" , code_h),
              ("person_prefer_h_3" , code_h),
              ("contents_attribute_h" , code_h),
              ("contents_attribute_l" , code_l),
]

# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [
    #("contents_attribute_a","person_attribute_a"),
    ("contents_attribute_c","person_prefer_c"),

    ("person_prefer_d_1_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_1_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_d" , "contents_attribute_d_attribute_d_d"),
    ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_d" , "contents_attribute_d_attribute_d_d"),

    ("person_prefer_h_2_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_2_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
    ("person_prefer_h_3_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_3_attribute_h_l" , "contents_attribute_h_attribute_h_l"),

]


In [ ]:
# 날짜형 변수 전환
train_data["contents_open_dt"]=pd.to_datetime(train_data["contents_open_dt"])
test_data["contents_open_dt"]=pd.to_datetime(test_data["contents_open_dt"])


# 날짜 피쳐 생성
for i in [train_data, test_data]:
    i['DAY'] = i['contents_open_dt'].dt.day
    #i['DAY_range']=i['DAY'].apply(lambda x: "초" if x<=10 else ("중" if x<=20 else "말"))
    i['WEEKDAY'] = i['contents_open_dt'].dt.weekday
    #i['WEEKDAY_range']=i['WEEKDAY'].apply(lambda x: "평일" if x<=4 else "주말")
    i['HOUR'] = i['contents_open_dt'].dt.hour
    #i['MONTH'] = i['contents_open_dt'].dt.month
    i['HOUR_range'] = pd.cut(i['contents_open_dt'], 4, labels = ['1', '2', '3', '4'], right = True, include_lowest=True)

#train=train.query("MONTH>=11")
#train.drop(labels="MONTH",axis=1,inplace=True)



#train.drop(labels="contents_open_dt",inplace=True, axis=1)
#test.drop(labels="contents_open_dt",inplace=True, axis=1)

# 학습에 필요없는 컬럼 리스트
cols_drop = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt", "contents_rn" ]

In [ ]:
x_train, y_train = preprocess_data(train_data, cols_merge = cols_merge , cols_equi= cols_equi , cols_drop = cols_drop)
x_test, _ = preprocess_data(test_data,is_train = False, cols_merge = cols_merge , cols_equi= cols_equi  , cols_drop = cols_drop)
x_train.shape , y_train.shape , x_test.shape

((501951, 74), (501951,), (46404, 74))

In [ ]:
# e 가 순서형 변수이기에 절댓값 마이너스를 했으나 오히려 성능 감소

x_train["e_minus"]=x_train["contents_attribute_e"]-x_train["person_prefer_e"];    x_train["e_minus"]=x_train["e_minus"].apply(lambda x: 1 if abs(x)<2 else 0)
x_test["e_minus"]=x_test["contents_attribute_e"]-x_test["person_prefer_e"];    x_test["e_minus"]=x_test["e_minus"].apply(lambda x: 1 if abs(x)<2 else 0)

In [ ]:
# 사람 별로 특징을 구분하여 그룹지은 걸 타겟 인코딩

x_train["person_group"]=x_train["person_attribute_a"].apply(str)+x_train["person_attribute_a_1"].apply(str)+x_train["person_attribute_b"].apply(str)
x_test["person_group"]=x_test["person_attribute_a"].apply(str)+x_test["person_attribute_a_1"].apply(str)+x_test["person_attribute_b"].apply(str)

x_train1=x_train.copy()
x_train1["target"]=y_train

group=pd.DataFrame(x_train1.groupby("person_group")["target"].agg(t="mean"))
x_train=pd.merge(x_train,group,how="left",on="person_group")
x_test=pd.merge(x_test,group,how="left",on="person_group")

x_train.drop(labels="person_group", axis=1, inplace=True)
x_test.drop(labels="person_group", axis=1, inplace=True)

In [ ]:
# 컨텐츠 d 의 잔차 합 피쳐

#x_train["d_minus_sum"]=x_train["contents_attribute_d"]*3-x_train["person_prefer_d_1"]-x_train["person_prefer_d_2"]-x_train["person_prefer_d_3"]
#x_test["d_minus_sum"]=x_test["contents_attribute_d"]*3-x_test["person_prefer_d_1"]-x_test["person_prefer_d_2"]-x_test["person_prefer_d_3"]

In [ ]:
x_train["user_a"]=np.sum(x_train[['person_attribute_a','person_attribute_a_1']].astype("str"),axis=1).astype(int)

x_test["user_a"]=np.sum(x_test[['person_attribute_a','person_attribute_a_1']].astype("str"),axis=1).astype(int)

x_train.drop(labels=['person_attribute_a','person_attribute_a_1'],axis=1,inplace=True)
x_test.drop(labels=['person_attribute_a','person_attribute_a_1'],axis=1,inplace=True)

In [ ]:
x_train=pd.get_dummies(x_train, columns=["HOUR_range"])
x_test=pd.get_dummies(x_test, columns=["HOUR_range"])

cat_features = x_train.columns[x_train.nunique() > 2].tolist()
cat_features.remove("t")

In [ ]:
x_train["match_sum_d"]=np.sum(x_train[[x for x in x_train.columns if "match" in x and "math_sum" not in x and "_d" in x]],axis=1)

x_test["match_sum_d"]=np.sum(x_test[[x for x in x_test.columns if "match" in x and "math_sum" not in x and "_d" in x]],axis=1)

x_train["match_sum_h"]=np.sum(x_train[[x for x in x_train.columns if "match" in x and "math_sum" not in x and "_h" in x]],axis=1)

x_test["match_sum_h"]=np.sum(x_test[[x for x in x_test.columns if "match" in x and "math_sum" not in x and "_h" in x]],axis=1)

In [ ]:
is_holdout = False
n_splits = 5
iterations = 1000
patience = 50

cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED )

def objective(trial):
  param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
      'task_type':trial.suggest_categorical('task_type', ['GPU'])
  }

  X_train1, X_valid1, y_train1, y_valid1 = train_test_split(x_train,y_train,test_size=0.2)
  
  cat = CatBoostClassifier(**param)
  cat.fit(X_train1, y_train1,
          eval_set=[(X_train1, y_train1), (X_valid1,y_valid1)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_pred = cat.predict_proba(X_valid1)
  score = f1_score(y_valid1, cat_pred)

  return score

sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    sampler = sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

In [ ]:
scores = []
models = []


models = []
for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []

    model = CatBoostClassifier(iterations=iterations,random_state=SEED,task_type="GPU",eval_metric="F1",cat_features=cat_features,one_hot_max_size=4)
    model.fit(x_train.iloc[tri], y_train[tri], 
            eval_set=[(x_train.iloc[vai], y_train[vai])], 
            early_stopping_rounds=patience ,
            verbose = 50
        )
    
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break    

Learning rate set to 0.042777
0:	learn: 0.6240141	test: 0.6243714	best: 0.6243714 (0)	total: 107ms	remaining: 1m 46s
50:	learn: 0.6424729	test: 0.6490673	best: 0.6490673 (50)	total: 5.79s	remaining: 1m 47s
100:	learn: 0.6543482	test: 0.6686408	best: 0.6686408 (100)	total: 12s	remaining: 1m 46s
150:	learn: 0.6594252	test: 0.6784297	best: 0.6784297 (150)	total: 18.1s	remaining: 1m 41s
200:	learn: 0.6630024	test: 0.6820740	best: 0.6820740 (200)	total: 24s	remaining: 1m 35s
250:	learn: 0.6655673	test: 0.6840630	best: 0.6840630 (250)	total: 30s	remaining: 1m 29s
300:	learn: 0.6679051	test: 0.6850528	best: 0.6850528 (300)	total: 35.8s	remaining: 1m 23s
350:	learn: 0.6701414	test: 0.6853808	best: 0.6855142 (346)	total: 41.5s	remaining: 1m 16s
400:	learn: 0.6718060	test: 0.6853950	best: 0.6857705 (373)	total: 47s	remaining: 1m 10s
450:	learn: 0.6729997	test: 0.6852246	best: 0.6859546 (420)	total: 52.5s	remaining: 1m 3s
bestTest = 0.6859546411
bestIteration = 420
Shrink model to first 421 itera

In [ ]:
print(scores)
print(np.mean(scores))
#0.6834

[0.6859546410504178, 0.6845863953269267, 0.6858138169976713, 0.6838737807941307, 0.6818177510306591]
0.6844092770399611


In [ ]:
threshold = 0.375

In [ ]:
pred_list = []
scores = []
for i,(tri, vai) in enumerate( cv.split(x_train) ):
    pred = models[i].predict_proba(x_train.iloc[vai])[:, 1]
    pred = np.where(pred >= threshold , 1, 0)
    score = f1_score(y_train[vai],pred)
    scores.append(score)
    pred = models[i].predict_proba(x_test)[:, 1]
    pred_list.append(pred)
print(scores)
print(np.mean(scores))

[0.7119507030525088, 0.7130125338540027, 0.712579415501906, 0.7127943329771658, 0.7123137441888187]
0.7125301459148804


In [ ]:
np.mean(scores)   # 단 돌릴때마다 어느정도 성능이 계속 바뀜, gpu를 사용해서 그런가?
#0.7129     -   기본 코드
#0.710           -   트레인 불러올 때 날자순 정렬 + 점수는 낮아졌지만 제출시 점수는 오히려 오름.  -> 과적합 방지 요소 +
#0.7109          -   e, c 코드 마이너스 피쳐 추가   성능 낮아짐    추후 추가 x
#0.7111          -   속성 정보마다 인코딩.   제출시 성능 매우 약간 개선
#0.7104          -   컨텐츠 d 정보를 유저 선호 d 에 잔차 합 피쳐 추가  성능 악화 제출은 안해봄. 추후 추가 x
#0.7106          - 최고 성능... 0.7111일 때와 같은 처리 , 랜덤 시드 고정 적용 
#0.7109          - 최고 성능... 0.7111일 때와 같은 처리 , 랜덤 시드 고정 적용  d_code전체
#0.7134          - early stopping 강화  -> 제출 시 떨어짐 추가 x
#0.7103          - train 을 4월 이후로 줄여봄.   -> 성능이 약간 떨어짐.  train 줄이기 금지.
#0.7105
#0.7111          - n_split=6 사용 x
#0.7126          - early-60  early 더 높이면 안될듯 함 splt이랑 사용 x
#0.7111          -# match전부해서 sum 추가
#0.7104          - 알고보니 e만 순서형 변수 e를 좀 유한 매치 코드 , c 매치코드 추가
#0.7113          - a매치는 점수를 오히려 낮추는 느낌도 존재   -> a 매치 제외시 점수 오름.
#0.7108          - math_sum을 구분지음.
#0.7097          - c_macth제외.
#0.7119          - c_macth포함에서부터 다시 시작 iteration 감소 제출시 0.7넘음  최고 점수 현.
#0.7122          - learning rate 0.3으로 높임 성능 감소
#0.71114-> 7125?          - a매치 합체시

0.7125301459148804

In [ ]:
pred = np.mean( pred_list , axis = 0 )
pred = np.where(pred >= threshold , 1, 0)

In [ ]:
sample_submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sample_submission['target'] = pred
sample_submission

,id,target
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
46399,46399,1
46400,46400,1
46401,46401,1
46402,46402,1


In [ ]:
sample_submission.to_csv(f"{DATA_PATH}prediction.csv", index=False)